# The objective
This analysis compares some investing scenarios, in particular:
 - Investing 10$ every opening day
 - Investing 50$ every 5 opening days
 - Investing 210$ every 21 opening days
 - Investing 1000$ every 100 opening days
 - investing 2500$ every 250 opening days
The studies will be conducted over 10 years choosing a random starting date

In [27]:
from datetime import date, timedelta
from simulator import Simulator
import numpy as np
from random import randint

In [28]:
class Config:
    def __init__(self, day_to_wait: int, amount_to_invest: float):
        self.day_to_wait = day_to_wait
        self.amount_to_invest = amount_to_invest

    def __str__(self):
        return f"Investing {self.amount_to_invest}$ every {self.day_to_wait} days"
    
configs = [
    Config(1, 10),
    Config(5,50),
    Config(20, 200),
    Config(100,1000),
    Config(250, 2500)
]

In [29]:

# get the ROI for a given config, node 2500 trading days are about 10 years in the nyse
def get_roi(config: Config, start_date: date, number_of_days: int = 2500) -> float:
    
    simulator = Simulator(start_date=start_date, end_date=None,start_cash=0)

    day_passed = 0

    while day_passed < number_of_days:
        for _ in range(config.day_to_wait):
            simulator.next_day()
        simulator.insert_funds(config.amount_to_invest)
        simulator.buy()
        #print("investing")

        day_passed += config.day_to_wait

    return simulator.get_current_roi()


In [30]:
Simulator.load_dataset()
START: date = Simulator.MIN_DATE
END: date = Simulator.MAX_DATE - timedelta(days=365*11)
DAY_DIFFERENCE: int = (END - START).days

def get_random_date() -> date:
    return START + timedelta(days=randint(0,DAY_DIFFERENCE))

In [31]:
# return average roi and standard deviation of roi
def calculate_average_roi(config: Config, start_days_to_test: list[date]) -> (float, float):
    roi_list = []
    for start_day in start_days_to_test:
        roi_list.append(get_roi(config, start_day))
    return np.average(roi_list), np.std(roi_list)

In [32]:
class Result:
    def __init__(self, config: Config, average_roi: float, std_dev: float):
        self.config = config
        self.average_roi = average_roi
        self.std_dev = std_dev
    
    def __str__(self):
        return f"{self.config}\nAverage ROI: {self.average_roi}\nStandard Deviation: {self.std_dev}"
    
def calculate_roi_for_config(config: Config, start_days_to_test: list[date]) -> Result:
    average_roi, std_dev = calculate_average_roi(config, start_days_to_test)
    return Result(config, average_roi, std_dev)

In [34]:
NUMBER_OF_TESTS = 10

days_to_test = [get_random_date() for _ in range(NUMBER_OF_TESTS)]

for config in configs:
    print(calculate_roi_for_config(config, days_to_test))
    print()

Investing 10$ every 1 days
Average ROI: 35.964000000000006
Standard Deviation: 22.89296057743515

Investing 50$ every 5 days
Average ROI: 36.001999999999995
Standard Deviation: 22.885885956195796

Investing 200$ every 20 days
Average ROI: 36.22500000000001
Standard Deviation: 22.825628249842325

Investing 1000$ every 100 days
Average ROI: 37.386
Standard Deviation: 22.59691890501889

Investing 2500$ every 250 days
Average ROI: 39.587
Standard Deviation: 24.144545988690698

